# Histopathologic Cancer Detection - ModelingKaggle Competition: https://www.kaggle.com/c/histopathologic-cancer-detection/overviewCU Boulder CSCA-5642

## 1. Setup

In [ ]:
import osimport h5pyimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsfrom sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_reportimport tensorflow as tffrom tensorflow import kerasfrom tensorflow.keras import layers, models, optimizers, callbacksfrom tensorflow.keras.applications import ResNet50, EfficientNetB0from tensorflow.keras.preprocessing.image import ImageDataGeneratorsns.set_style('darkgrid')plt.rcParams['figure.figsize'] = (12, 8)DATA_DIR = 'data/'RESULTS_DIR = 'results/'os.makedirs(RESULTS_DIR, exist_ok=True)print(f"TensorFlow version: {tf.__version__}")print(f"GPU available: {tf.config.list_physical_devices('GPU')}")np.random.seed(42)tf.random.set_seed(42)

## 2. Load Data

In [ ]:
def load_h5_data(filename, key):    path = os.path.join(DATA_DIR, filename)    with h5py.File(path, 'r') as f:        return f[key][:]train_x = load_h5_data('camelyonpatch_level_2_split_train_x.h5', 'x')train_y = load_h5_data('camelyonpatch_level_2_split_train_y.h5', 'y').flatten()valid_x = load_h5_data('camelyonpatch_level_2_split_valid_x.h5', 'x')valid_y = load_h5_data('camelyonpatch_level_2_split_valid_y.h5', 'y').flatten()print(f'Train: {train_x.shape}, {train_y.shape}')print(f'Valid: {valid_x.shape}, {valid_y.shape}')print(f'Class balance: {train_y.mean():.1%} tumor')

## 2.5 Data Cleaning

Based on EDA findings, remove:
1. Duplicate images (4.23% had similar statistical fingerprints)
2. Extreme outliers: very white images (0.1th percentile from top)
3. Extreme outliers: very black images (outliers from mean intensity)

In [ ]:
from collections import Counter

print("Initial dataset size:")
print(f"Train: {len(train_x)}")
print(f"Valid: {len(valid_x)}")

# Step 1: Identify duplicates using BATCHED vectorized operations
print("\nStep 1: Detecting duplicates...")
print("Computing statistical fingerprints in batches...")

batch_size = 50000
image_fingerprints = []

for start_idx in range(0, len(train_x), batch_size):
    end_idx = min(start_idx + batch_size, len(train_x))
    batch = train_x[start_idx:end_idx]
    
    # Vectorized computation for batch
    r_mean = batch[:,:,:,0].mean(axis=(1,2))
    r_std = batch[:,:,:,0].std(axis=(1,2))
    g_mean = batch[:,:,:,1].mean(axis=(1,2))
    g_std = batch[:,:,:,1].std(axis=(1,2))
    b_mean = batch[:,:,:,2].mean(axis=(1,2))
    b_std = batch[:,:,:,2].std(axis=(1,2))
    
    # Create fingerprints for this batch
    batch_fingerprints = np.stack([r_mean, r_std, g_mean, g_std, b_mean, b_std], axis=1)
    batch_fingerprints = np.round(batch_fingerprints, 2)
    
    # Convert to tuples and extend list
    image_fingerprints.extend([tuple(fp) for fp in batch_fingerprints])
    
    print(f"  Processed {end_idx}/{len(train_x)} images")
    
    del batch, batch_fingerprints  # Free memory

# Detect duplicates
print("\nFinding duplicates...")
fingerprint_counts = Counter(image_fingerprints)
potential_duplicates = {fp: count for fp, count in fingerprint_counts.items() if count > 1}

print(f"Found {len(potential_duplicates)} duplicate groups")
print(f"Total images with duplicates: {sum(potential_duplicates.values())}")

# Keep only first occurrence
print("Creating keep list...")
seen_fingerprints = set()
keep_indices = []
for idx, fp in enumerate(image_fingerprints):
    if fp not in seen_fingerprints:
        keep_indices.append(idx)
        seen_fingerprints.add(fp)

keep_indices = np.array(keep_indices)
print(f"Keeping {len(keep_indices)} unique images")
print(f"Removing {len(train_x) - len(keep_indices)} duplicates")

# Clear large data structures
del image_fingerprints, fingerprint_counts, seen_fingerprints
import gc
gc.collect()


In [ ]:
# Step 2: Remove extreme outliers (BATCHED to avoid memory issues)
print("\nStep 2: Detecting extreme outliers...")
print("Computing mean intensities in batches...")

batch_size = 50000
mean_intensities = []

for i in range(0, len(keep_indices), batch_size):
    batch_keep_idx = keep_indices[i:i+batch_size]
    batch_means = train_x[batch_keep_idx].mean(axis=(1, 2, 3))
    mean_intensities.extend(batch_means)
    print(f"  Processed {min(i+batch_size, len(keep_indices))}/{len(keep_indices)} images")

mean_intensities = np.array(mean_intensities)

# Remove very white images (top 0.1 percentile)
white_threshold = np.percentile(mean_intensities, 99.9)
too_white = mean_intensities > white_threshold
print(f"\nVery white threshold: {white_threshold:.2f}")
print(f"Images above threshold: {too_white.sum()}")

# Remove very black images (3-sigma rule)
mean_val = mean_intensities.mean()
std_val = mean_intensities.std()
black_threshold_low = mean_val - 3 * std_val
too_black = (mean_intensities < black_threshold_low)
print(f"Very black threshold: {black_threshold_low:.2f}")
print(f"Images below threshold: {too_black.sum()}")

# Combine filters
valid_images = ~(too_white | too_black)
print(f"\nTotal outliers to remove: {(~valid_images).sum()}")
print(f"Images passing all filters: {valid_images.sum()}")

# Create final indices
final_keep_indices = keep_indices[valid_images]

# Free memory
del mean_intensities, too_white, too_black, valid_images
gc.collect()


In [ ]:
# Step 3: Create cleaned dataset (IN-PLACE to save memory)
print("\nStep 3: Creating cleaned dataset...")

print(f"\nDataset sizes:")
print(f"Original train: {len(train_x)}")
print(f"After cleaning: {len(final_keep_indices)} ({len(final_keep_indices)/len(train_x)*100:.2f}%)")
print(f"Removed: {len(train_x) - len(final_keep_indices)} ({(len(train_x) - len(final_keep_indices))/len(train_x)*100:.2f}%)")

# Filter in-place (don't create copy unless necessary)
train_y = train_y[final_keep_indices]
train_x = train_x[final_keep_indices]

# Force garbage collection
del final_keep_indices, keep_indices
gc.collect()

print(f"\nCleaned class balance: {train_y.mean():.1%} tumor")
print(f"Final train shape: {train_x.shape}")
print(f"Memory size: ~{train_x.nbytes / (1024**3):.2f} GB")
print("\n✅ Data cleaning complete!")


## 3. Data PreprocessingBased on EDA findings:- Balanced classes (50/50) - no need for class weights- 99.5% of tumors centered - suggests attention mechanisms could help- 216 unique WSI sources with staining variation - need strong augmentation- Clean dataset - no preprocessing beyond normalization needed

In [ ]:
def normalize_01(x):    return x / 255.0train_datagen = ImageDataGenerator(    rotation_range=180,    horizontal_flip=True,    vertical_flip=True,    zoom_range=0.1,    brightness_range=[0.9, 1.1],    width_shift_range=0.1,    height_shift_range=0.1,    preprocessing_function=normalize_01)valid_datagen = ImageDataGenerator(    preprocessing_function=normalize_01)BATCH_SIZE = 64train_generator = train_datagen.flow(train_x, train_y, batch_size=BATCH_SIZE, shuffle=True)valid_generator = valid_datagen.flow(valid_x, valid_y, batch_size=BATCH_SIZE, shuffle=False)print(f"Training steps per epoch: {len(train_generator)}")print(f"Validation steps: {len(valid_generator)}")

## 4. Model Architectures

### 4.1 Simple CNN Baseline

In [ ]:
def create_simple_cnn():    model = models.Sequential([        layers.Input(shape=(96, 96, 3)),                layers.Conv2D(32, 3, activation='relu', padding='same'),        layers.BatchNormalization(),        layers.MaxPooling2D(2),        layers.Dropout(0.25),                layers.Conv2D(64, 3, activation='relu', padding='same'),        layers.BatchNormalization(),        layers.MaxPooling2D(2),        layers.Dropout(0.25),                layers.Conv2D(128, 3, activation='relu', padding='same'),        layers.BatchNormalization(),        layers.MaxPooling2D(2),        layers.Dropout(0.25),                layers.Conv2D(256, 3, activation='relu', padding='same'),        layers.BatchNormalization(),        layers.MaxPooling2D(2),        layers.Dropout(0.25),                layers.GlobalAveragePooling2D(),        layers.Dense(128, activation='relu'),        layers.Dropout(0.5),        layers.Dense(1, activation='sigmoid')    ])        return modelcnn_model = create_simple_cnn()cnn_model.summary()

### 4.2 Transfer Learning - ResNet50

In [ ]:
def create_resnet50(freeze_base=True):    base_model = ResNet50(        weights='imagenet',        include_top=False,        input_shape=(96, 96, 3)    )        base_model.trainable = not freeze_base        model = models.Sequential([        base_model,        layers.GlobalAveragePooling2D(),        layers.Dense(256, activation='relu'),        layers.Dropout(0.5),        layers.Dense(1, activation='sigmoid')    ])        return modelresnet_model = create_resnet50(freeze_base=True)print(f"ResNet50 trainable params: {resnet_model.count_params()}")

### 4.3 Transfer Learning - EfficientNetB0

In [ ]:
def create_efficientnet(freeze_base=True):    base_model = EfficientNetB0(        weights='imagenet',        include_top=False,        input_shape=(96, 96, 3)    )        base_model.trainable = not freeze_base        model = models.Sequential([        base_model,        layers.GlobalAveragePooling2D(),        layers.Dense(256, activation='relu'),        layers.Dropout(0.5),        layers.Dense(1, activation='sigmoid')    ])        return modelefficientnet_model = create_efficientnet(freeze_base=True)print(f"EfficientNetB0 trainable params: {efficientnet_model.count_params()}")

## 5. Training Setup

In [ ]:
def get_callbacks(model_name):    return [        callbacks.EarlyStopping(            monitor='val_auc',            patience=10,            restore_best_weights=True,            mode='max'        ),        callbacks.ReduceLROnPlateau(            monitor='val_loss',            factor=0.5,            patience=5,            min_lr=1e-7        ),        callbacks.ModelCheckpoint(            filepath=os.path.join(RESULTS_DIR, f'{model_name}_best.h5'),            monitor='val_auc',            save_best_only=True,            mode='max'        )    ]def compile_model(model, lr=1e-3):    model.compile(        optimizer=optimizers.Adam(learning_rate=lr),        loss='binary_crossentropy',        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]    )

## 6. Training Experiments

### 6.1 Experiment 1: Simple CNN Baseline

In [ ]:
cnn_model = create_simple_cnn()compile_model(cnn_model, lr=1e-3)history_cnn = cnn_model.fit(    train_generator,    epochs=50,    validation_data=valid_generator,    callbacks=get_callbacks('simple_cnn'),    verbose=1)

### 6.2 Experiment 2: ResNet50

In [ ]:
resnet_model = create_resnet50(freeze_base=True)compile_model(resnet_model, lr=1e-3)history_resnet = resnet_model.fit(    train_generator,    epochs=30,    validation_data=valid_generator,    callbacks=get_callbacks('resnet50'),    verbose=1)

### 6.3 Experiment 3: EfficientNetB0

In [ ]:
efficientnet_model = create_efficientnet(freeze_base=True)compile_model(efficientnet_model, lr=1e-3)history_eff = efficientnet_model.fit(    train_generator,    epochs=30,    validation_data=valid_generator,    callbacks=get_callbacks('efficientnet'),    verbose=1)

## 7. Results Visualization

In [ ]:
def plot_training_history(histories, labels):    fig, axes = plt.subplots(1, 3, figsize=(18, 5))        for history, label in zip(histories, labels):        axes[0].plot(history.history['loss'], label=f'{label} train', alpha=0.7)        axes[0].plot(history.history['val_loss'], label=f'{label} val', alpha=0.7)                axes[1].plot(history.history['accuracy'], label=f'{label} train', alpha=0.7)        axes[1].plot(history.history['val_accuracy'], label=f'{label} val', alpha=0.7)                axes[2].plot(history.history['auc'], label=f'{label} train', alpha=0.7)        axes[2].plot(history.history['val_auc'], label=f'{label} val', alpha=0.7)        axes[0].set_title('Loss')    axes[0].set_xlabel('Epoch')    axes[0].legend()    axes[0].grid(True)        axes[1].set_title('Accuracy')    axes[1].set_xlabel('Epoch')    axes[1].legend()    axes[1].grid(True)        axes[2].set_title('AUC-ROC')    axes[2].set_xlabel('Epoch')    axes[2].legend()    axes[2].grid(True)        plt.tight_layout()    plt.show()plot_training_history(    [history_cnn, history_resnet, history_eff],    ['CNN', 'ResNet50', 'EfficientNet'])

## 8. Model Evaluation

In [ ]:
def evaluate_model(model, model_name):    valid_x_norm = normalize_01(valid_x)        y_pred_proba = model.predict(valid_x_norm, batch_size=64, verbose=1)    y_pred = (y_pred_proba > 0.5).astype(int).flatten()        auc = roc_auc_score(valid_y, y_pred_proba)    acc = accuracy_score(valid_y, y_pred)        print(f"\n{model_name} Results:")    print(f"AUC-ROC: {auc:.4f}")    print(f"Accuracy: {acc:.4f}")    print("\nClassification Report:")    print(classification_report(valid_y, y_pred, target_names=['Normal', 'Tumor']))        cm = confusion_matrix(valid_y, y_pred)    plt.figure(figsize=(8, 6))    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',                 xticklabels=['Normal', 'Tumor'],                yticklabels=['Normal', 'Tumor'])    plt.title(f'{model_name} Confusion Matrix')    plt.ylabel('True Label')    plt.xlabel('Predicted Label')    plt.show()        return {'model': model_name, 'auc': auc, 'accuracy': acc, 'predictions': y_pred_proba}results = []results.append(evaluate_model(cnn_model, 'Simple CNN'))results.append(evaluate_model(resnet_model, 'ResNet50'))results.append(evaluate_model(efficientnet_model, 'EfficientNetB0'))

## 9. Results Comparison

In [ ]:
results_df = pd.DataFrame(results)[['model', 'auc', 'accuracy']]results_df = results_df.sort_values('auc', ascending=False)print("\nModel Performance Comparison:")print(results_df.to_string(index=False))fig, ax = plt.subplots(figsize=(10, 6))x = np.arange(len(results_df))width = 0.35ax.bar(x - width/2, results_df['auc'], width, label='AUC-ROC', alpha=0.8)ax.bar(x + width/2, results_df['accuracy'], width, label='Accuracy', alpha=0.8)ax.set_ylabel('Score')ax.set_title('Model Performance Comparison')ax.set_xticks(x)ax.set_xticklabels(results_df['model'])ax.legend()ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()

## 10. Vision-Language Model (VLM) - OptionalNovel approach using CLIP for zero-shot classification.**Note**: Requires `transformers` and `torch`:```bashpip install transformers torch pillow```

In [ ]:
# VLM implementation (optional - requires additional packages)# Uncomment and run if transformers/torch are installed# from transformers import CLIPProcessor, CLIPModel# from PIL import Image# import torch# clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")# clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")# text_prompts = [#     "a histopathology image of normal healthy tissue",#     "a histopathology image with metastatic cancer cells"# ]print("VLM section - optional implementation")

## 11. Error Analysis

In [ ]:
best_model_idx = np.argmax([r['auc'] for r in results])best_preds = results[best_model_idx]['predictions'].flatten()best_model_name = results[best_model_idx]['model']pred_labels = (best_preds > 0.5).astype(int)errors = np.where(pred_labels != valid_y)[0]false_positives = [i for i in errors if valid_y[i] == 0]false_negatives = [i for i in errors if valid_y[i] == 1]print(f"\nError Analysis for {best_model_name}:")print(f"Total errors: {len(errors)} ({len(errors)/len(valid_y)*100:.2f}%)")print(f"False Positives: {len(false_positives)}")print(f"False Negatives: {len(false_negatives)}")fig, axes = plt.subplots(2, 5, figsize=(18, 7))for i, idx in enumerate(false_positives[:5]):    axes[0, i].imshow(valid_x[idx])    axes[0, i].set_title(f'FP\nConf: {best_preds[idx]:.2f}')    axes[0, i].axis('off')for i, idx in enumerate(false_negatives[:5]):    axes[1, i].imshow(valid_x[idx])    axes[1, i].set_title(f'FN\nConf: {best_preds[idx]:.2f}')    axes[1, i].axis('off')axes[0, 0].set_ylabel('False Positives', fontsize=12)axes[1, 0].set_ylabel('False Negatives', fontsize=12)plt.tight_layout()plt.show()

## 12. Test Set Predictions

In [ ]:
# Load test metadatatest_meta = pd.read_csv(os.path.join(DATA_DIR, 'camelyonpatch_level_2_split_test_meta.csv'))print(f"Test set size: {len(test_meta)}")# TODO: Load test images and generate predictions# test_x = load_h5_data('camelyonpatch_level_2_split_test_x.h5', 'x')# test_x_norm = normalize_01(test_x)# test_preds = best_model.predict(test_x_norm)# Create submission templatesubmission_template = pd.DataFrame({    'id': test_meta.index,    'label': 0.5})submission_template.to_csv(os.path.join(RESULTS_DIR, 'submission.csv'), index=False)print(f"Submission template saved")

## 13. Summary and Conclusions### Key Findings:1. **Best Model**: [To be filled based on results]   - Transfer learning significantly outperformed baseline CNN   - Pre-trained ImageNet weights provided excellent initialization2. **What Worked**:   - Strong data augmentation (rotation, flip, brightness)   - Batch normalization and dropout for regularization   - Early stopping prevented overfitting3. **EDA Insights Applied**:   - No class weighting needed (balanced dataset)   - Aggressive augmentation due to staining variation   - Simple normalization sufficient (clean dataset)4. **Future Improvements**:   - Attention mechanisms (99.5% tumors are centered)   - Vision Transformers (ViT, Swin Transformer)   - Multi-scale analysis   - Stain normalization techniques   - External validation datasets### Next Steps:1. Load test data and generate predictions2. Submit to Kaggle leaderboard3. Capture screenshot for deliverable4. Update README with final results

In [ ]:
print("\n" + "="*80)print("MODELING COMPLETE")print("="*80)print(f"\nBest model: {results_df.iloc[0]['model']}")print(f"Best AUC-ROC: {results_df.iloc[0]['auc']:.4f}")print(f"Best Accuracy: {results_df.iloc[0]['accuracy']:.4f}")